## Imports and Parameters

In [2]:
import tensorflow as tf
from tensorflow import keras
import pickle
import os
import numpy as np
from NozPiker_Funcs import main as NZ
import pandas as pd
import glob

In [10]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Define Data Path
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()),'data/build_data/')

# Fetching pdbs and converting to images

In [12]:
MRC_PATHS = glob.glob(DATA_DIR + '*mrcs')

In [13]:
len(MRC_PATHS)

1

In [14]:
store = {}

for MRC in MRC_PATHS:
    size = NZ.GetImageSet(MRC,MRC.split('/')[-1].split('_')[0],store)

In [15]:
train_images, train_labels, class_names = NZ.CreateTrainingData(store,size)

## Create Model

In [19]:
# Define model
def create_model():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(size,size)),
        keras.layers.Dense(128,activation='relu'),
        keras.layers.Dense(len(class_names))
    ])
    
    model.compile(optimizer = 'adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    return model


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 238144)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               30482560  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 30,482,689
Trainable params: 30,482,689
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 238144)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               30482560  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 30,482,689
Trainable params: 30,482,689
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [1]:
checkpoint_path = os.path.join(os.path.dirname(os.path.dirname(DATA_DIR)),'checkpoints/cp.ckpt')
checkpoint_dir = os.path.dirname(checkpoint_path)

NameError: name 'os' is not defined

In [22]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [24]:
model.fit(train_images,
          train_labels,
          epochs=10,
          callbacks=[cp_callback])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00001: saving model to training_1/cp.ckpt
1/1 [==============================] - 0s 454ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00002: saving model to training_1/cp.ckpt
1/1 [==============================] - 9s 9s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00003: saving model to training_1/cp.ckpt
1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00004: saving model to training_1/cp.ckpt
1/1 [==============================] - 5s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.0

In [32]:
os.path.dirname(os.path.dirname(DATA_DIR))

'/home/paul/Projects/NozPiker/data'

In [33]:
# Save Model as Hd5
MODEL_DIR = os.path.join(os.path.dirname(os.path.dirname(DATA_DIR)),'models')
model.save(os.path.join(MODEL_DIR,'Initial_Train.h5'))

## Load Trained Model

In [36]:
# Load the model
model = keras.models.load_model(os.path.join(MODEL_DIR,'Initial_Train.h5'))